<h2><b>Langchain : Loader </b></h2>

In [1]:
# loader --> Text
from langchain_community.document_loaders import TextLoader
loader = TextLoader("./data/rou.txt")
text_doc = loader.load()
# text_doc

In [2]:
# loader --> Web
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                       bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                           class_=("post-title","post-content","post-header")
                       )))
text_doc = loader.load()
# text_doc

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# loader --> PDF
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('./data/pdf_data.pdf')

text_doc = loader.load()
# text_doc

In [5]:
# Vector Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
db = Chroma.from_documents(text_doc,embedding_model)

C:\Users\aksha\AppData\Local\Temp\ipykernel_748\3945808333.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


<h2><b>Prompts | Chain | Retriver </b></h2>


In [6]:
# LLM 
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv() 

llm = ChatGoogleGenerativeAI(model = "gemini-1.5-pro")

In [7]:
# Chatprompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following Question based on the provided context.
    Think step by step before providing a detailed answer.
    I will tip you $1000 if the user finds the answer helpful
    <context>
    {context}
    </context>
    Question : {input}""")

prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    Answer the following Question based on the provided context.\n    Think step by step before providing a detailed answer.\n    I will tip you $1000 if the user finds the answer helpful\n    <context>\n    {context}\n    </context>\n    Question : {input}'), additional_kwargs={})])

In [ ]:
#Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

document_chain = create_stuff_documents_chain(llm,prompt)
retriever =db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever , document_chain)

In [ ]:
res= retrieval_chain.invoke({"input":" As in 2021, New Mexico had the highest rate of public health insurance coverage?"})
res['answer']

'Yes, the text states: "As in 2021, New Mexico had the highest rate of public health insurance coverage at 51.2 percent".  It goes on to compare this with Utah, which had the lowest rate at 22.2 percent.'